In [31]:
# 랜덤포레스트라이브러리
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#geopandas
import geopandas as gpd
import pandas as pd

# 공간연산
from shapely.geometry import Point


In [32]:
# 등대 SHP 파일 읽기
shp_file_lighthouse = "./data/해양수산부 국립해양조사원_등대_20240717/VI_LNDMRK_P.shp"
lighthouse = gpd.read_file(shp_file_lighthouse)

In [33]:
print(lighthouse.crs)

PROJCS["Korea_2000_Korea_Unified_Coordinate_System",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127.5],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",1000000],PARAMETER["false_northing",2000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [34]:
# lighthouse는 GeoDataFrame
# 원하는 컬럼 리스트
columns_to_keep = ['XMIN', 'YMIN', 'MNYMD']

# 원하는 컬럼만 선택하여 lighthouse 업데이트
lighthouse = lighthouse[columns_to_keep]

print(lighthouse.head())


           XMIN          YMIN     MNYMD
0  1.170227e+06  1.724263e+06  20231208
1  1.142759e+06  1.947584e+06  20231208
2  9.415726e+05  1.600094e+06  20231208
3  1.137052e+06  1.965146e+06  20231208
4 -1.758740e+06 -1.798592e+06  20231208


In [35]:
import geopandas as gpd
from shapely.geometry import Point

# GeoDataFrame으로 변환 (현재 lighthouse는 좌표계 정보가 없을 가능성 있음)
lighthouse['geometry'] = lighthouse.apply(lambda row: Point(row.XMIN, row.YMIN), axis=1)
lighthouse = gpd.GeoDataFrame(lighthouse, geometry='geometry')

# 현재 좌표계 설정 (Korea Unified Coordinate System, EPSG:5179로 추정)
lighthouse.set_crs(epsg=5179, inplace=True)  # EPSG:5179은 한국 통합 좌표계

# 좌표계를 EPSG:4326 (WGS84, 위경도)로 변환
lighthouse = lighthouse.to_crs(epsg=4326)

# 변환된 좌표를 x, y 컬럼에 저장
lighthouse['x'] = lighthouse.geometry.x  # 경도
lighthouse['y'] = lighthouse.geometry.y  # 위도

# 필요 없는 geometry 컬럼 삭제
lighthouse = lighthouse.drop(columns=['geometry', 'XMIN', 'YMIN'])

# 결과 확인
print(lighthouse.head())


      MNYMD           x          y
0  20231208  129.376835  35.499688
1  20231208  129.115405  37.516512
2  20231208  126.864374  34.392943
3  20231208  129.054136  37.675611
4  20231208  103.408297   3.361819
